In [ ]:
import calcbench as cb
import pandas as pd
import requests

In [ ]:
cb.api_client._rig_for_testing("localhost")

In [ ]:
def get_holdings(file_name: str):
    holdings = pd.read_csv(
        file_name, header=0
    )  # From https://www.barchart.com/etfs-funds/quotes/QQQ/constituents
    holdings = holdings[:-1]
    holdings.Symbol = holdings.Symbol.replace({"GOOGL": "GOOG", "BRK.B": "BRK"})
    holdings = holdings.rename({"Symbol": "ticker"}, axis=1)
    holdings = holdings.set_index("ticker")
    holdings = holdings["% Holding"].str.rstrip("%").astype("float") * 0.01
    holdings = holdings[holdings.index.dropna()]
    holdings = (
        holdings.groupby("ticker").sum().sort_values(ascending=False)
    )  # remove duplicates
    return holdings

In [ ]:
QQQ_holdings = get_holdings("./etf-constituents-06-02-2022 QQQ.csv")
SPY_holdings = get_holdings("./etf-constituents-06-02-2022 SPY.csv")

In [5]:
# Metrics order as they appear on an income statement.
income_statement_metrics = [
    "Revenue",
    "CostOfRevenue",
    "GrossProfit",
    "SGAExpense",
    "OperatingExpenses",
    "OperatingIncome",
    "EBIT",
    "InterestExpense",
    "IncomeTaxes",
    "NetIncome",
]

In [6]:
company_identifiers = SPY_holdings.index.union(QQQ_holdings.index).unique().unique()

In [9]:
d = cb.standardized(
    metrics=income_statement_metrics,
    company_identifiers=company_identifiers,
    period_type=cb.api_query_params.PeriodType.Annual,
    revisions=cb.api_query_params.Revisions.MostRecent,
)

In [112]:
def build_portfolio_income_statement(d: pd.DataFrame, holdings):
    by_fund_weight = (
        d[["value"]].unstack(["metric", "fiscal_period"]).mul(holdings, axis=0)
    )
    totals = by_fund_weight.sum()
    percent_of_revenue = totals / totals.loc[:, "Revenue"]
    horizontal = percent_of_revenue.unstack("fiscal_period").sort_index(
        axis=1, ascending=False
    )  # Sort columns by fiscal year descending.
    return horizontal.loc['value'].loc[income_statement_metrics]

In [113]:
x = build_portfolio_income_statement(d, QQQ_holdings)

In [114]:
x

fiscal_period,2027-0,2025-0,2024-0,2023-0,2022-0,2021-0,2020-0,2019-0,2018-0,2017-0,2016-0,2015-0,2014-0,2013-0,2012-0,2011-0,2010-0,2009-0,2008-0,2007-0
metric,,,,,,,,,,,,,,,,,,,,
Revenue,NaN,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
CostOfRevenue,NaN,0.000000,NaN,0.517830,0.502434,0.505716,0.528963,0.523855,0.527549,0.525537,0.522903,0.517620,0.517056,0.521154,0.488776,0.483330,0.465824,0.451322,0.452219,0.535117
GrossProfit,NaN,0.000000,NaN,0.442018,0.497566,0.494284,0.471002,0.476145,0.472450,0.474463,0.477092,0.478662,0.477451,0.477568,0.507700,0.513449,0.534176,0.548678,0.547781,0.464883
SGAExpense,NaN,0.044726,NaN,0.091296,0.116697,0.112144,0.122774,0.130310,0.125485,0.130558,0.134589,0.131820,0.143579,0.143111,0.143421,0.158342,0.173219,0.155898,0.194886,0.156006
OperatingExpenses,NaN,0.067566,NaN,0.205696,0.261386,0.244897,0.265692,0.269499,0.259061,0.247845,0.245979,0.245201,0.242194,0.238907,0.249007,0.241050,0.257515,0.250103,0.299250,0.266139
OperatingIncome,NaN,0.085829,NaN,0.276497,0.236180,0.249320,0.204840,0.205351,0.210930,0.213952,0.225655,0.230915,0.235530,0.243296,0.265949,0.265169,0.268548,0.248984,0.241680,0.198813
EBIT,NaN,0.058697,NaN,0.283975,0.231265,0.272855,0.222072,0.224212,0.234567,0.229035,0.234380,0.237626,0.241920,0.249607,0.274400,0.273583,0.274700,0.254827,0.249322,0.219663
InterestExpense,NaN,0.018787,NaN,0.001085,0.006837,0.007506,0.009718,0.011514,0.011717,0.010798,0.008878,0.006169,0.005339,0.004743,0.004503,0.004895,0.005294,0.005477,0.005549,0.007503
IncomeTaxes,NaN,0.015545,NaN,0.045304,0.033385,0.038397,0.032461,0.030175,0.050921,0.063687,0.052512,0.059230,0.057083,0.055679,0.065708,0.059515,0.066187,0.066501,0.067602,0.058497
